In [43]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
cd '/content/drive/Shared drives/Rahul_Mohit/ECE202A/WALG/'

/content/drive/Shared drives/Rahul_Mohit/ECE202A/WALG


Create a new folder in your Drive. (I created a folder named HAR) Copy the data files and the 2 pyrhon files to that folder (utils.py and existing_models.py). Then change working directory to that folder.

In [46]:
from collections import Counter
import sys
import time
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras.models import load_model
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score, accuracy_score
import matplotlib.pyplot as plt
import itertools
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, LSTM, Dense, Dropout, Flatten
from tensorflow.keras.layers import BatchNormalization, Permute, Reshape

In [47]:
## Function to reshape data as required by the mnodel.
def reshape_data(X_tr, X_va, X_tst, network_type):
    _, win_len, dim = X_tr.shape

    if network_type == 'CNN' or network_type == 'ConvLSTM':
        # make it into (frame_number, dimension, window_size, channel=1) for convNet
        X_tr = np.swapaxes(X_tr, 1, 2)
        X_va = np.swapaxes(X_va, 1, 2)
        X_tst = np.swapaxes(X_tst, 1, 2)

        X_tr = np.reshape(X_tr, (-1, dim, win_len, 1))
        X_va = np.reshape(X_va, (-1, dim, win_len, 1))
        X_tst = np.reshape(X_tst, (-1, dim, win_len, 1))

    elif network_type == 'MLP':
        X_tr = np.reshape(X_tr, (-1, dim * win_len))
        X_va = np.reshape(X_va, (-1, dim * win_len))
        X_tst = np.reshape(X_tst, (-1, dim * win_len))

    return X_tr, X_va, X_tst

## Function to generate sliding windows.
def sliding_window(frame_length, step, X, Y, tstamps_all):
  X_f = []
  Y_f = []
  tstamp = []
  for loop in range(0,X.shape[0]-frame_length,step):
    # X_f.append(X[loop: loop+frame_length]);
    tstamp.append(tstamps_all[loop])
    X_f.append(
      np.vstack((
        X[loop: loop+frame_length],
        np.mean(X[loop: loop+frame_length], axis=0),
        np.std(X[loop: loop+frame_length], axis=0, dtype=np.float64)
      ))
    );
    Y_f.append(Y[loop]);
  return X_f, Y_f, tstamp

## Function to store data as numpy files.
def numpify_and_store(X, y, X_name, y_name, outdatapath, shuffle=False):
      X = np.array(X)
      y = np.array(y)
      # Shuffle the train set
      if shuffle is True:
          np.random.seed(123)
          neworder = np.random.permutation(X.shape[0])
          X = X[neworder, :, :]
          y = y[neworder]
      # Save binary file
      xpath = os.path.join(outdatapath, X_name)
      ypath = os.path.join(outdatapath, y_name)
      np.save(xpath, X)
      np.save(ypath, y)
      print('Storing ' + xpath, ypath)

In [48]:
# Variables
frame_length  = int(200);
step          = 40;
network_type  = "CNN";
model_name    = 'final_CNN_1608149269_golden_random.hdf5';

file_name     = '2020_12_16_Suparno_All';
tstamp_file_name = './Data/Live_Data/tstamps/'+file_name+'_tstamp.csv';

ACTIVITIES_MAP = {
    0: 'Walking',
    1: 'Stairs',
    2: 'Sitting',
    3: 'Brushing Teeth',
    4: 'Eating',
    5: 'Jogging',
    6: 'Clapping'
}

# Data Preprocessing

* Read CSV
* Split data into windows
* Add Mean & Std. Dev. in windows
* Save processed data as numpy files.
* Store timestamps of the windows as CSV file.

In [49]:
file_path = './Data/Live_Data/raw/'+file_name+'.csv'
raw_data = pd.read_csv(file_path)
formatted_data = pd.DataFrame(
    {
        "acc_x":   raw_data.loc[:,'accelerometerAccelerationX(G)'],
        "acc_y":   raw_data.loc[:,'accelerometerAccelerationY(G)'],
        "acc_z":   raw_data.loc[:,'accelerometerAccelerationZ(G)'],
        "gyro_x":  raw_data.loc[:,'motionRotationRateX(rad/s)'],
        "gyro_y":  raw_data.loc[:,'motionRotationRateY(rad/s)'],
        "gyro_z":  raw_data.loc[:,'motionRotationRateZ(rad/s)'],
        "label":   raw_data.loc[:,'label']
    }
)
tstamps_all = raw_data.loc[:,'loggingTime(txt)']
x_np = formatted_data.iloc[:,0:formatted_data.shape[1]-1].to_numpy();
y_np = formatted_data['label'].to_numpy();
x_psw_np, y_psw_np, tstamp = sliding_window(frame_length, step, x_np, y_np, tstamps_all);
numpify_and_store(
  x_psw_np, 
  y_psw_np, 
  X_name='X_'+file_name, 
  y_name='Y_'+file_name, 
  outdatapath='./Data/Live_Data/processed/', 
  shuffle=False 
);
np.savetxt(
    './Data/Live_Data/tstamps/'+file_name+'_tstamp.csv',  
    tstamp, 
    delimiter =", ",
    fmt = "%s"
);

Storing ./Data/Live_Data/processed/X_2020_12_16_Suparno_All ./Data/Live_Data/processed/Y_2020_12_16_Suparno_All


# Inference

* Load trained model.
* Predicit activities at different time stamps.
* Post-process predictions.
* Print processed predictions with timestamps. 

In [50]:
model = load_model('./Models/WISDM_PAMAP2/'+model_name)

In [51]:
X_data0 = np.load('./Data/Live_Data/processed/X_'+file_name+'.npy')
Y_true = np.load('./Data/Live_Data/processed/Y_'+file_name+'.npy')
tstamp_data = pd.read_csv(tstamp_file_name, header=None)
X_data, a, b = reshape_data(X_data0, X_data0, X_data0, network_type)
Y_pred = np.argmax(model.predict(X_data), axis=1)
last_act1 = Y_pred[0]
last_act2 = Y_pred[0]
last_act3 = Y_pred[0]
last_printed = -1
print("===========================================");
print("       DATE      TIME     ACTIVITY")
print("===========================================");
for x in range(len(Y_pred)):
  last_act1 = last_act2
  last_act2 = last_act3
  last_act3 = Y_pred[x]
  if ((x==0) | ((last_act3 == Y_pred[x]) & (last_act3 == last_act2) & (last_act2 == last_act1) & (last_printed != Y_pred[x]))):
    last_printed = Y_pred[x]
    print("    {}   {}".format(tstamp_data.iloc[x,0][:-10], ACTIVITIES_MAP[Y_pred[x]]))
print("===========================================");

       DATE      TIME     ACTIVITY
    2020-12-16 17:39:31   Walking
    2020-12-16 17:40:51   Eating
    2020-12-16 17:41:07   Sitting
    2020-12-16 17:41:43   Eating
    2020-12-16 17:42:37   Sitting
    2020-12-16 17:42:55   Eating
    2020-12-16 17:44:29   Brushing Teeth
